In [1]:
import sys
sys.path.append('../craw-data')
from kafka import KafkaProducer
import requests
from json import dumps
import time
from kafka import KafkaConsumer

In [2]:
from get_detaill_item import get_detail_product
from get_item_and_shop_ids import get_item_shop_info

In [3]:
import pandas as pd
categoryList = pd.read_csv('../craw-data/raw-data/category_list.csv')
categoryList.head()

,catid,parent_catid,name,display_name,image,unselected_image,selected_image,level,children,block_buyer_platform
0,11035567,0,Men Clothes,Thời Trang Nam,687f3967b7c2fe6a134a2c11894eea4b,fe2d27d140e1f34855c1bfe3168c1b58,9131b98c529e7ccfffc36193432aa927,1,NaN,NaN
1,11035639,0,Women Clothes,Thời Trang Nữ,75ea42f9eca124e9cb3cde744c060e4d,293ca6f400001a38f477ef1c8ebe2831,7002b396e4d22be3decacceeef762265,1,NaN,NaN
2,11036030,0,Mobile & Gadgets,Điện Thoại & Phụ Kiện,31234a27876fb89cd522d7e3db1ba5ca,94fb4fb95a8598e8d31f0853f904c15b,070098d9b3cf1a1d35be2a6ef1bb3cd0,1,NaN,NaN
3,11036194,0,"Moms, Kids & Babies",Mẹ & Bé,099edde1ab31df35bc255912bab54a5e,f91ae14aebd454762ab61136ebcec4e0,9b56c2810b51f8a017b76c3cefc84daf,1,NaN,NaN
4,11036132,0,Consumer Electronics,Thiết Bị Điện Tử,978b9e4cb61c611aaaf58664fae133c5,4a6a4d8230cf961e18636fbf91bad4b5,067bd7ff52bd0d3700d9b0af722e83c9,1,NaN,NaN


In [4]:
def get_data(raw_dict, list_attribute):
    output = {}
    for att in list_attribute:
        output[att] = raw_dict[att]
    return output

In [9]:
from kafka import KafkaProducer
kafka_producer = KafkaProducer(bootstrap_servers='localhost:9092', value_serializer=lambda x: dumps(x).encode('utf-8'))
# from a - b category
a = 26
b = 27
list_attributes = ["itemid", "shopid", "price_before_discount", "price", "discount", "historical_sold", "sold", "name", "item_status", "item_rating", "liked_count", "cmt_count", "item_type", "shop_location", "global_sold", "shop_vouchers", "categories"]
for item in range(a,b):
    data = []
    topic = categoryList.catid.iloc[item]
    print(f'topic {item} "{categoryList.display_name.iloc[item]}" is runing...')
    for newest in range(0,8001,60): 
        item_list_id = get_item_shop_info(topic,newest)
        data.extend(get_detail_product(item_list_id,topic))
    message_data = list(map(lambda x: get_data(x,list_attributes),data))
    for it in message_data:
        print("publishing to kafka...")
        kafka_producer.send('item-shopee', value=it)



topic 26 "Voucher & Dịch Vụ" is runing...
An exception occurred with match_id 11035898 and newest 0 


AttributeError: 'NoneType' object has no attribute 'shape'

In [6]:
# Bán chạy nhất(số lượng bán nhiều nhất) (theo category)
# Doanh thu cao nhất (theo category)
# Discount nhiều nhất (theo category)
# Rating (theo product)
# Phân tích số lượng sản phẩm bán được theo location
# Loại mặt hàng có nhiều tương tác nhất (dựa vào số like, số comment)


# itemid, shopid, price_before_discount, price, discount, historical_sold, sold, name, item_status, item_rating, liked_count, cmt_count, item_type, shop_location, global_sold, shop_vouchers, categories